# Pointers to alternative and more advanced methods

These alternative and more advanced packages mostly try to address the following problem:

Suppose that somebody has written some code in a function `f`, and we would like to access that code *after the fact*. How much can we recover about the function?

In [1]:
f(x, y) = 2x + y + 1

f (generic function with 1 method)

## Tracing with Symbolics.jl

For certain functions that are "not too complicated", a powerful technique has recently become available using the `Symbolics.jl` package: we **trace** the function with **symbolic variables**. This gives us a symbolic version of the function that we can then manipulate in a similar way to Julia's `Expr`.

In [10]:
using Symbolics

In [11]:
@variables x, y

2-element Vector{Num}:
 x
 y

In [12]:
ex = f(x, y)

1 + y + 2x

We have recovered the mathematical expression corresponding to the function definition!

In [13]:
typeof(ex)

Num

We can get information about the structure of the expression:

In [14]:
Symbolics.get_variables(ex)

2-element Vector{Any}:
 y
 x

In [15]:
Symbolics.arguments(Symbolics.value(ex))

3-element Vector{Any}:
 1
  y
  2x

The internal structure is more complicated, in a way, than a Julia `Expr`: 

In [17]:
dump(ex, maxdepth=2)

Num
  val: SymbolicUtils.Add{Real, Int64, Dict{Any, Number}, Nothing}
    coeff: Int64 1
    dict: Dict{Any, Number}
    sorted_args_cache: Base.RefValue{Any}
    hash: Base.RefValue{UInt64}
    metadata: Nothing nothing


## Lowered form an Intermediate Representation (IR)

Julia unfortunately does not store the source code of the function definition. It does make accessible a **lowered** version, where the code has been rewritten into a simpler (for the compiler!) form:

In [14]:
codeinfo = @code_lowered f(3, 4)

CodeInfo(
1 ─ %1 = 2 * x
│   %2 = %1 + y + 1
└──      return %2
)

In [15]:
codeinfo.code

3-element Vector{Any}:
 :(2 * _2)
 :(%1 + _3 + 1)
 :(return %2)

## Cassette.jl

Cassette provides the possibility of making so-called **contexts**, within which it is possible to redefine the meaning of given functions using multiple dispatch.

## IRTools.jl

IRTools works with a lower-level Intermediate Representation (IR) of the code, similar to that which is produced by `@code_typed`


In [27]:
codeinfo = @code_typed f(3, 4)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = Base.add_int(%1, y)::Int64
│   %3 = Base.add_int(%2, 1)::Int64
└──      return %3
) => Int64

In [31]:
code = first(codeinfo).code

4-element Vector{Any}:
 :(Base.mul_int(2, _2))
 :(Base.add_int(%1, _3))
 :(Base.add_int(%2, 1))
 :(return %3)

Working with a lower-level IR has the advantage that you can extract the IR for a given function directly and then modify it. It has the disadvantage that the code is much lower level.

## MLStyle.jl

[Note: ML here refers to the functional programming language, not to "Machine Learning".]

`MLStyle.jl` provides high-performance pattern matching that makes it easier to match and extract pieces of ASTs.
`MacroTools.jl` is an older package with some similar functionality.